In [ ]:
%gui qt

import copy
import numpy as np

from pyqtgraph.Qt import QtGui, QtCore

from plottr.data import datadict as dd
from plottr.node import node
from plottr.apps.tools import make_sequential_flowchart, make_sequential_flowchart_with_gui

In [87]:
def err(*x):
    raise ValueError(x)

class AutoNodeGuiTemplate(node.NodeWidget):
    
    optionToNode = QtCore.pyqtSignal(object)
    allOptionsToNode = QtCore.pyqtSignal(object)
    
    def __init__(self, parent=None):
        super().__init__(parent)
        self.layout = QtGui.QFormLayout()
        self.setLayout(self.layout)
        
        self.optGetters = {}
        self.optSetters = {}
    
    def getAllOptions(self):
        ret = {}
        for n, f in self.optGetters.items():
            ret[n] = f()
            
        return ret
    
    def addOption(self, name, specs, confirm=True):
        optType = specs.get('type', None)
        widget = None
        
        if optType == int:
            widget = QtGui.QSpinBox()
            widget.setValue(specs.get('initialValue', 1))
            if not confirm:
                widget.valueChanged.connect(lambda x: self.optionChangeWrapper(name, x))
            self.optGetters[name] = widget.value
            self.optSetters[name] = widget.setValue
            
        elif optType == float:
            widget = QtGui.QDoubleSpinBox()
            widget.setValue(specs.get('initialValue', 1))
            if not confirm:
                widget.valueChanged.connect(lambda x: self.optionChangeWrapper(name, x))
            self.optGetters[name] = widget.value
            self.optSetters[name] = widget.setValue
            
        elif optType == str:
            widget = QtGui.QLineEdit()
            widget.setText(specs.get('initialValue', ''))
            if not confirm:
                widget.textChanged.connect(lambda x: self.optionChangeWrapper(name, x))
            self.optGetters[name] = widget.text
            self.optSetters[name] = widget.setText
            
        if widget is not None:
            self.layout.addRow(name, widget)
            
    
    def addConfirm(self):
        widget = QtGui.QPushButton('Confirm')
        widget.pressed.connect(self.allOptionsWrapper)
        self.layout.addRow('', widget)
    
    @node.NodeWidget.emitGuiUpdate('optionToNode')
    def optionChangeWrapper(self, name, value):
        return name, value
    
    @node.NodeWidget.emitGuiUpdate('allOptionsToNode')
    def allOptionsWrapper(self):
        return self.getAllOptions()
    
    @node.NodeWidget.updateGuiFromNode
    def setOptionFromNode(self, opt, value):
        self.optSetters[opt](value)
        

class AutoNodeTemplate(node.Node):
    
    def processOptionUpdate(self, optName, value):
        if self.ui is not None and optName in self.ui.optSetters:
            self.ui.optSetters[optName](value)
    
    def __init__(self, name):
        super().__init__(name)
        
        self.opts = []

        self.ui.optionToNode.connect(self.setOptionFromUi)
        self.ui.allOptionsToNode.connect(self.setAllOptionsFromUi)
    
    def addOption(self, name, specs):
        # create setter and getter function
        varname = '_'+name
        setattr(self, varname, specs.get("initialValue", None))
        
        def getter(self):
            return getattr(self, varname)
        
        @node.Node.updateOption(name)
        def setter(self, val):
            setattr(self, varname, val)
        
        setattr(self.__class__, name, property(getter, setter))
        
        self.opts.append(name)
        
    def setOptionFromUi(self, nameAndVal):
        name, val = nameAndVal
        setattr(self, name, val)
        
    def setAllOptionsFromUi(self, opts):
        for opt, val in opts.items():
            setattr(self, opt, val)    
        

def autoNode(nodeName, confirm=True, **options):
    
    def decorator(func):
        
        class AutoNodeGui(AutoNodeGuiTemplate):
            
            optionChanged = QtCore.pyqtSignal(str, object)
            
            def __init__(self, parent=None):
                super().__init__(parent)
                for optName, optSpecs in options.items():
                    self.addOption(optName, optSpecs, confirm=confirm)
                
                if confirm:
                    self.addConfirm()
                    
        
        class AutoNode(AutoNodeTemplate):
            def __init__(self, name):
                super().__init__(name)
                for optName, optSpecs in options.items():
                    self.addOption(optName, optSpecs)
        
        AutoNode.__name__ = nodeName
        AutoNode.nodeName = nodeName
        AutoNode.autoNodeOptions = options
        AutoNode.process = func
        
        AutoNode.useUi = True
        AutoNode.uiClass = AutoNodeGui
            
        return AutoNode
    
    return decorator


@autoNode('Scaling',
    confirm=True,
    scalingFactor={'initialValue' : 1, 'type' : int},
    offset={'initialValue' : 0, 'type' : float},
    label={'initialValue' : 'scaled', 'type' : str},
)
def scaling(self, **data):
    data = data['dataIn']
    if data is None:
        return None

    data = copy.deepcopy(data)
    for d in data.dependents():
        data[d]['values'] -= self.offset
        data[d]['values'] *= self.scalingFactor
        if self.label != '':
            data[d+'_'+self.label] = data.pop(d)
        
    return dict(dataOut=data)

In [88]:
nodes, fc, win = make_sequential_flowchart_with_gui([scaling])
scalingNode = nodes[0]


inputData = dd.DataDict(
    x = dict(values=np.linspace(0, 10, 26), unit='A'),
    y = dict(values=np.sin(np.linspace(0, 10, 26)), unit='B', axes=['x'])
)

fc.setInput(dataIn=inputData)

In [95]:
fc.output()

{'dataOut': {'x': {'values': array([ 0. ,  0.4,  0.8,  1.2,  1.6,  2. ,  2.4,  2.8,  3.2,  3.6,  4. ,
           4.4,  4.8,  5.2,  5.6,  6. ,  6.4,  6.8,  7.2,  7.6,  8. ,  8.4,
           8.8,  9.2,  9.6, 10. ]), 'unit': 'A'},
  'y_meh': {'values': array([ 0.        ,  1.16825503,  2.15206827,  2.79611726,  2.99872081,
           2.72789228,  2.02638954,  1.00496445, -0.17512243, -1.32756133,
          -2.27040749, -2.85480622, -2.98849383, -2.65036397, -1.89379991,
          -0.83824649,  0.34964761,  1.48234005,  2.38100359,  2.90375902,
           2.96807474,  2.56379672,  1.75475158,  0.66866974, -0.52298034,
          -1.63206333]),
   'unit': 'B',
   'axes': ['x']}}}

In [92]:
scalingNode.scalingFactor = 3

In [94]:
scalingNode.label = 'meh'